In [1]:
number_list = list(range(1, 28))
formatted_list = ', c'.join(map(str, number_list))
print(formatted_list)

1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14, c15, c16, c17, c18, c19, c20, c21, c22, c23, c24, c25, c26, c27


In [2]:
import configparser
config = configparser.ConfigParser()
config.read('config.txt')

source_folder = config.get('Configuration', 'source_folder')
time_step = int(config.get('Configuration', 'time_step'))
window_size = int(config.get('Configuration', 'window_size'))
data_folder = config.get('Configuration', 'data_folder')
ini_date = config.get('Configuration', 'ini_date')
end_date = config.get('Configuration', 'end_date')

users=config.get('Configuration', 'users')
users = [(item.strip()) for item in users.split(',')]

scenes=config.get('Configuration', 'scenes')
scenes = [(item.strip()) for item in scenes.split(',')]

room=config.get('Configuration', 'room')

In [3]:
import numpy as np
from pandas import read_csv    
import pandas as pd
import time

from datetime import datetime

off_zone=60*60*2

def day_time(ti):
    return int((int)((ti+off_zone)/(60*60*24)))


def time2str(tt):
    return datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")


t0 = (int)(datetime.strptime(ini_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (ini_date,"Init date is", t0, "day:",day_time(t0), time2str(t0))

tN = (int)(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (end_date,"End date is", tN, "day:",day_time(tN), time2str(tN))


def day_time0(ti):
    return day_time(ti)-day_time(t0)+1

def relT(ti):
    return (int)((ti-t0)/time_step)

days=list(range(day_time(t0),day_time(tN)+1))
print(days)

2024-03-16 21:31:18 Init date is 1710621078 day: 19798 2024-03-16 21:31:18
2024-03-16 21:41:04 End date is 1710621664 day: 19798 2024-03-16 21:41:04
[19798]


In [4]:
def getLoc(room,s):
    print("S:",s)
    xy=config.get(room, s)
    xy = [(item.strip()) for item in xy.split(',')]
    xy = [list(map(int, item.split('x'))) for item in xy]
    return xy

loc_sensor={}

fp_sensors = config.get(room, 'maps')
fp_sensors = [(item.strip()) for item in fp_sensors.split(',')]

w = int(config.get(room, 'w'))
h = int(config.get(room, 'h'))

for fp in fp_sensors:
    loc_sensor[fp]=getLoc(room,fp)
print(loc_sensor)

S: c1
S: c2
S: c3
S: c4
S: c5
S: c6
S: c7
S: c8
S: c9
S: c10
S: c11
S: c12
S: c13
S: c14
S: c15
S: c16
S: c17
S: c18
S: c19
S: c20
S: c21
S: c22
S: c23
S: c24
S: c25
S: c26
S: c27
S: c28
S: c29
S: c30
{'c1': [[0, 0], [0, 1]], 'c2': [[2, 0]], 'c3': [[3, 0]], 'c4': [[4, 0]], 'c5': [[5, 0]], 'c6': [[1, 1]], 'c7': [[2, 1]], 'c8': [[3, 1]], 'c9': [[4, 1]], 'c10': [[5, 0], [5, 1]], 'c11': [[0, 2], [0, 3]], 'c12': [[1, 2]], 'c13': [[2, 2]], 'c14': [[3, 2]], 'c15': [[4, 2]], 'c16': [[1, 3]], 'c17': [[2, 3]], 'c18': [[3, 3]], 'c19': [[4, 3]], 'c20': [[5, 2], [5, 3]], 'c21': [[0, 4], [0, 5]], 'c22': [[4, 1]], 'c23': [[4, 2]], 'c24': [[4, 3]], 'c25': [[4, 4]], 'c26': [[5, 1]], 'c27': [[5, 2]], 'c28': [[5, 3]], 'c29': [[5, 4]], 'c30': [[5, 4], [5, 5]]}


In [5]:
df_total_map = {}
for d in days:
    df_total_map[d]={}
    for fp in fp_sensors:
        df_total_map[d][fp]=None
        for s in scenes:
            print(d,fp,source_folder+"/"+s+"/binary/"+str(d)+"."+fp+".tsv")
            try:
                series=read_csv(source_folder+"/"+s+"/binary/"+str(d)+"."+fp+".tsv", sep="\t",parse_dates=True, header=None, names=["time","date","state"])
                print(d,fp,series.size)
            except:
                print("Not data")
                continue
            if(df_total_map[d][fp] is None):
                df_total_map[d][fp]=series
            else:
                df_total_map[d][fp] = df_total_map[d][fp].append(series, ignore_index=True)



19798 c1 ./data/kitchen/2-person-labelled/Scene1/binary/19798.c1.tsv
19798 c1 6
19798 c1 ./data/kitchen/2-person-labelled/Scene2/binary/19798.c1.tsv
19798 c1 6
19798 c2 ./data/kitchen/2-person-labelled/Scene1/binary/19798.c2.tsv
19798 c2 3
19798 c2 ./data/kitchen/2-person-labelled/Scene2/binary/19798.c2.tsv
19798 c2 15
19798 c3 ./data/kitchen/2-person-labelled/Scene1/binary/19798.c3.tsv
19798 c3 9
19798 c3 ./data/kitchen/2-person-labelled/Scene2/binary/19798.c3.tsv
19798 c3 27
19798 c4 ./data/kitchen/2-person-labelled/Scene1/binary/19798.c4.tsv
19798 c4 9
19798 c4 ./data/kitchen/2-person-labelled/Scene2/binary/19798.c4.tsv
19798 c4 9
19798 c5 ./data/kitchen/2-person-labelled/Scene1/binary/19798.c5.tsv
19798 c5 9
19798 c5 ./data/kitchen/2-person-labelled/Scene2/binary/19798.c5.tsv
19798 c5 15
19798 c6 ./data/kitchen/2-person-labelled/Scene1/binary/19798.c6.tsv
19798 c6 24
19798 c6 ./data/kitchen/2-person-labelled/Scene2/binary/19798.c6.tsv
19798 c6 27
19798 c7 ./data/kitchen/2-person-la

C:\Users\Usuario\AppData\Local\Temp\ipykernel_28172\1656400201.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[d][fp] = df_total_map[d][fp].append(series, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_28172\1656400201.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[d][fp] = df_total_map[d][fp].append(series, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_28172\1656400201.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[d][fp] = df_total_map[d][fp].append(series, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_28172\1656400201.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in

19798 c20 27
19798 c21 ./data/kitchen/2-person-labelled/Scene1/binary/19798.c21.tsv
19798 c21 3
19798 c21 ./data/kitchen/2-person-labelled/Scene2/binary/19798.c21.tsv
19798 c21 21
19798 c22 ./data/kitchen/2-person-labelled/Scene1/binary/19798.c22.tsv
19798 c22 15
19798 c22 ./data/kitchen/2-person-labelled/Scene2/binary/19798.c22.tsv
19798 c22 21
19798 c23 ./data/kitchen/2-person-labelled/Scene1/binary/19798.c23.tsv
19798 c23 3
19798 c23 ./data/kitchen/2-person-labelled/Scene2/binary/19798.c23.tsv
19798 c23 3
19798 c24 ./data/kitchen/2-person-labelled/Scene1/binary/19798.c24.tsv
19798 c24 21
19798 c24 ./data/kitchen/2-person-labelled/Scene2/binary/19798.c24.tsv
19798 c24 27
19798 c25 ./data/kitchen/2-person-labelled/Scene1/binary/19798.c25.tsv
19798 c25 9
19798 c25 ./data/kitchen/2-person-labelled/Scene2/binary/19798.c25.tsv
19798 c25 18
19798 c26 ./data/kitchen/2-person-labelled/Scene1/binary/19798.c26.tsv
19798 c26 3
19798 c26 ./data/kitchen/2-person-labelled/Scene2/binary/19798.c26.t

C:\Users\Usuario\AppData\Local\Temp\ipykernel_28172\1656400201.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[d][fp] = df_total_map[d][fp].append(series, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_28172\1656400201.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[d][fp] = df_total_map[d][fp].append(series, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_28172\1656400201.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[d][fp] = df_total_map[d][fp].append(series, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_28172\1656400201.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in

In [6]:
import cv2
import os

def getStateTime(df_total,tt):
    df_intervalo1 = df_total[(df_total['time'] >= (tt))]
    df_intervalo2 = df_total[(df_total['time'] <= (tt))]
    #print("tt",tt,time2str(tt))
    #print("df_intervalo1",df_intervalo1, len(df_intervalo1))
    #print("df_intervalo2",df_intervalo2, len(df_intervalo2))
    if(len(df_intervalo2)==0):
        return 0
    if(df_intervalo2.iloc[-1]['state']=='on'):
        return 1
    return 0
    #print("df_intervalo2",df_intervalo2.iloc[-1])
    
    return 1
    
for d in days:
    print(d)
    os.makedirs(data_folder+"/"+room+"/DAY_"+str(d)+"/", exist_ok=True)
    for tt in range(t0,tN,time_step):
        print(tt,day_time(tt),time2str(tt))
        #print(tt-off_zone,day_time(tt-off_zone),time2str(tt-off_zone))
        if(day_time(tt) != d):
            continue

            
        img0=np.zeros((h, w))
        for fp in fp_sensors:
            df_total=df_total_map[d][fp]    
            #df_intervalo = df_total[(df_total['time'] >= (tt-window_size/2)) & (df_total['time'] <= (tt)) & (df_total['state']=='on')]
            #print("DF",df_intervalo)            
            #for index, row in df_total.iterrows():
            #    dif=tt-row['time']+1
            #    print("@TIME",tt,fp,row, row['time'], "dif:", dif)
            #    for xy in loc_sensor[fp]:
            #        img0[xy[0]][xy[1]]=img0[xy[0]][xy[1]]+int(255/dif)
            for ti in range(int(tt-window_size/2),tt+1):
                #print(ti)
                v=getStateTime(df_total,ti)*0.25
                dif=tt-ti+1
                #print("@TIMA ",tt,v, "difA:", dif)
                for xy in loc_sensor[fp]:
                    img0[xy[0]][xy[1]]=img0[xy[0]][xy[1]]+int(v*255/dif)
                    
            for ti in range(tt+1, int(tt+1+window_size/2)):
                v=getStateTime(df_total,ti)*0.25
                dif=ti-tt+1
                #print("@TIMB ",tt,v, "difB:", dif)
                for xy in loc_sensor[fp]:
                    img0[xy[0]][xy[1]]=img0[xy[0]][xy[1]]+int(v*255/dif)
                    
        img0[img0 > 255] = 255
        cv2.imwrite(data_folder+"/"+room+"/DAY_"+str(d)+"/FP."+str(tt)+".png", img0) 

19798
1710621078 19798 2024-03-16 21:31:18
1710621079 19798 2024-03-16 21:31:19
1710621080 19798 2024-03-16 21:31:20
1710621081 19798 2024-03-16 21:31:21
1710621082 19798 2024-03-16 21:31:22
1710621083 19798 2024-03-16 21:31:23
1710621084 19798 2024-03-16 21:31:24
1710621085 19798 2024-03-16 21:31:25
1710621086 19798 2024-03-16 21:31:26
1710621087 19798 2024-03-16 21:31:27
1710621088 19798 2024-03-16 21:31:28
1710621089 19798 2024-03-16 21:31:29
1710621090 19798 2024-03-16 21:31:30
1710621091 19798 2024-03-16 21:31:31
1710621092 19798 2024-03-16 21:31:32
1710621093 19798 2024-03-16 21:31:33
1710621094 19798 2024-03-16 21:31:34
1710621095 19798 2024-03-16 21:31:35
1710621096 19798 2024-03-16 21:31:36
1710621097 19798 2024-03-16 21:31:37
1710621098 19798 2024-03-16 21:31:38
1710621099 19798 2024-03-16 21:31:39
1710621100 19798 2024-03-16 21:31:40
1710621101 19798 2024-03-16 21:31:41
1710621102 19798 2024-03-16 21:31:42
1710621103 19798 2024-03-16 21:31:43
1710621104 19798 2024-03-16 21:3

1710621302 19798 2024-03-16 21:35:02
1710621303 19798 2024-03-16 21:35:03
1710621304 19798 2024-03-16 21:35:04
1710621305 19798 2024-03-16 21:35:05
1710621306 19798 2024-03-16 21:35:06
1710621307 19798 2024-03-16 21:35:07
1710621308 19798 2024-03-16 21:35:08
1710621309 19798 2024-03-16 21:35:09
1710621310 19798 2024-03-16 21:35:10
1710621311 19798 2024-03-16 21:35:11
1710621312 19798 2024-03-16 21:35:12
1710621313 19798 2024-03-16 21:35:13
1710621314 19798 2024-03-16 21:35:14
1710621315 19798 2024-03-16 21:35:15
1710621316 19798 2024-03-16 21:35:16
1710621317 19798 2024-03-16 21:35:17
1710621318 19798 2024-03-16 21:35:18
1710621319 19798 2024-03-16 21:35:19
1710621320 19798 2024-03-16 21:35:20
1710621321 19798 2024-03-16 21:35:21
1710621322 19798 2024-03-16 21:35:22
1710621323 19798 2024-03-16 21:35:23
1710621324 19798 2024-03-16 21:35:24
1710621325 19798 2024-03-16 21:35:25
1710621326 19798 2024-03-16 21:35:26
1710621327 19798 2024-03-16 21:35:27
1710621328 19798 2024-03-16 21:35:28
1

1710621524 19798 2024-03-16 21:38:44
1710621525 19798 2024-03-16 21:38:45
1710621526 19798 2024-03-16 21:38:46
1710621527 19798 2024-03-16 21:38:47
1710621528 19798 2024-03-16 21:38:48
1710621529 19798 2024-03-16 21:38:49
1710621530 19798 2024-03-16 21:38:50
1710621531 19798 2024-03-16 21:38:51
1710621532 19798 2024-03-16 21:38:52
1710621533 19798 2024-03-16 21:38:53
1710621534 19798 2024-03-16 21:38:54
1710621535 19798 2024-03-16 21:38:55
1710621536 19798 2024-03-16 21:38:56
1710621537 19798 2024-03-16 21:38:57
1710621538 19798 2024-03-16 21:38:58
1710621539 19798 2024-03-16 21:38:59
1710621540 19798 2024-03-16 21:39:00
1710621541 19798 2024-03-16 21:39:01
1710621542 19798 2024-03-16 21:39:02
1710621543 19798 2024-03-16 21:39:03
1710621544 19798 2024-03-16 21:39:04
1710621545 19798 2024-03-16 21:39:05
1710621546 19798 2024-03-16 21:39:06
1710621547 19798 2024-03-16 21:39:07
1710621548 19798 2024-03-16 21:39:08
1710621549 19798 2024-03-16 21:39:09
1710621550 19798 2024-03-16 21:39:10
1

In [7]:
scene_user = {}
for d in days:
    for user in users:
        scene_user[user] = {}
        for s in scenes:
            print(d,source_folder+"/"+s+"/location_gt_"+user+"/"+str(d)+".location.tsv")
            try:
                series=read_csv(source_folder+"/"+s+"/location_gt_"+user+"/"+str(d)+".location.tsv", sep="\t",parse_dates=True, header=None, names=["time","date","time2","x","y"])
                print(d,series.size,(series["time"].iloc[0],series["time"].iloc[-1]))
                scene_user[user][s] = (series["time"].iloc[0],series["time"].iloc[-1])
                print(scene_user[user][s])
            except Exception as e:
                print("Not data",e)
                continue
def getScene(t0):
    for user, user_scene in scene_user.items():
        for scene, times in user_scene.items():
            #print(t0,times[0],times[1])
            if(t0>times[0]-window_size/2 and t0<times[1]+window_size/2):
                return scene
    return None

19798 ./data/kitchen/2-person-labelled/Scene1/location_gt_ed9c/19798.location.tsv
19798 1215 (1710621088, 1710621359)
(1710621088, 1710621359)
19798 ./data/kitchen/2-person-labelled/Scene2/location_gt_ed9c/19798.location.tsv
19798 1380 (1710621360, 1710621664)
(1710621360, 1710621664)
19798 ./data/kitchen/2-person-labelled/Scene1/location_gt_867f/19798.location.tsv
19798 1145 (1710621078, 1710621359)
(1710621078, 1710621359)
19798 ./data/kitchen/2-person-labelled/Scene2/location_gt_867f/19798.location.tsv
19798 1255 (1710621362, 1710621664)
(1710621362, 1710621664)


In [8]:
from PIL import Image

def getFPImg(day,tt):
    return np.array(Image.open(data_folder+"/"+room+"/DAY_"+str(day)+"/FP."+str(tt)+".png"))

mean_scenes={}
for d in days:
    for tt in range(t0,tN,1):
        print(tt,day_time(tt))
        if(day_time(tt) != d):
            continue
                
        scene=getScene(tt)
        print("Scene",scene)
        if not scene:
            continue
        if scene not in mean_scenes:
            mean_scenes[scene]=[]
        imgX=getFPImg(d,tt)
        print("imgX",imgX.shape)
        mean_scenes[scene].append(imgX)
        
          

1710621078 19798
Scene Scene1
imgX (6, 6)
1710621079 19798
Scene Scene1
imgX (6, 6)
1710621080 19798
Scene Scene1
imgX (6, 6)
1710621081 19798
Scene Scene1
imgX (6, 6)
1710621082 19798
Scene Scene1
imgX (6, 6)
1710621083 19798
Scene Scene1
imgX (6, 6)
1710621084 19798
Scene Scene1
imgX (6, 6)
1710621085 19798
Scene Scene1
imgX (6, 6)
1710621086 19798
Scene Scene1
imgX (6, 6)
1710621087 19798
Scene Scene1
imgX (6, 6)
1710621088 19798
Scene Scene1
imgX (6, 6)
1710621089 19798
Scene Scene1
imgX (6, 6)
1710621090 19798
Scene Scene1
imgX (6, 6)
1710621091 19798
Scene Scene1
imgX (6, 6)
1710621092 19798
Scene Scene1
imgX (6, 6)
1710621093 19798
Scene Scene1
imgX (6, 6)
1710621094 19798
Scene Scene1
imgX (6, 6)
1710621095 19798
Scene Scene1
imgX (6, 6)
1710621096 19798
Scene Scene1
imgX (6, 6)
1710621097 19798
Scene Scene1
imgX (6, 6)
1710621098 19798
Scene Scene1
imgX (6, 6)
1710621099 19798
Scene Scene1
imgX (6, 6)
1710621100 19798
Scene Scene1
imgX (6, 6)
1710621101 19798
Scene Scene1
imgX

imgX (6, 6)
1710621289 19798
Scene Scene1
imgX (6, 6)
1710621290 19798
Scene Scene1
imgX (6, 6)
1710621291 19798
Scene Scene1
imgX (6, 6)
1710621292 19798
Scene Scene1
imgX (6, 6)
1710621293 19798
Scene Scene1
imgX (6, 6)
1710621294 19798
Scene Scene1
imgX (6, 6)
1710621295 19798
Scene Scene1
imgX (6, 6)
1710621296 19798
Scene Scene1
imgX (6, 6)
1710621297 19798
Scene Scene1
imgX (6, 6)
1710621298 19798
Scene Scene1
imgX (6, 6)
1710621299 19798
Scene Scene1
imgX (6, 6)
1710621300 19798
Scene Scene1
imgX (6, 6)
1710621301 19798
Scene Scene1
imgX (6, 6)
1710621302 19798
Scene Scene1
imgX (6, 6)
1710621303 19798
Scene Scene1
imgX (6, 6)
1710621304 19798
Scene Scene1
imgX (6, 6)
1710621305 19798
Scene Scene1
imgX (6, 6)
1710621306 19798
Scene Scene1
imgX (6, 6)
1710621307 19798
Scene Scene1
imgX (6, 6)
1710621308 19798
Scene Scene1
imgX (6, 6)
1710621309 19798
Scene Scene1
imgX (6, 6)
1710621310 19798
Scene Scene1
imgX (6, 6)
1710621311 19798
Scene Scene1
imgX (6, 6)
1710621312 19798
Scene

imgX (6, 6)
1710621500 19798
Scene Scene2
imgX (6, 6)
1710621501 19798
Scene Scene2
imgX (6, 6)
1710621502 19798
Scene Scene2
imgX (6, 6)
1710621503 19798
Scene Scene2
imgX (6, 6)
1710621504 19798
Scene Scene2
imgX (6, 6)
1710621505 19798
Scene Scene2
imgX (6, 6)
1710621506 19798
Scene Scene2
imgX (6, 6)
1710621507 19798
Scene Scene2
imgX (6, 6)
1710621508 19798
Scene Scene2
imgX (6, 6)
1710621509 19798
Scene Scene2
imgX (6, 6)
1710621510 19798
Scene Scene2
imgX (6, 6)
1710621511 19798
Scene Scene2
imgX (6, 6)
1710621512 19798
Scene Scene2
imgX (6, 6)
1710621513 19798
Scene Scene2
imgX (6, 6)
1710621514 19798
Scene Scene2
imgX (6, 6)
1710621515 19798
Scene Scene2
imgX (6, 6)
1710621516 19798
Scene Scene2
imgX (6, 6)
1710621517 19798
Scene Scene2
imgX (6, 6)
1710621518 19798
Scene Scene2
imgX (6, 6)
1710621519 19798
Scene Scene2
imgX (6, 6)
1710621520 19798
Scene Scene2
imgX (6, 6)
1710621521 19798
Scene Scene2
imgX (6, 6)
1710621522 19798
Scene Scene2
imgX (6, 6)
1710621523 19798
Scene

In [9]:
for key, value in mean_scenes.items():
    mean_scenes[key]=np.array(value)
    mean_scenes[key] = np.median(mean_scenes[key], axis=0) 
    print(mean_scenes[key])
    print(f"Key: {key}, Value: {mean_scenes[key].shape}")  

[[167. 167. 167. 167.   0.   0.]
 [  0. 167. 167.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0. 167. 167.   0.   0.]]
Key: Scene1, Value: (6, 6)
[[167. 167.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.  52.  21.  42.   0.]]
Key: Scene2, Value: (6, 6)


In [10]:
for d in days:
    for key, value in mean_scenes.items():
        cv2.imwrite(data_folder+"/"+room+"/DAY_"+str(d)+"/FP.mean."+str(key)+".png", value)
    